#Customizing Models and Training Algorithms

#Custom Loss Functions

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Nadam

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28)) / 255.0
test_images = test_images.reshape((10000, 28 * 28)) / 255.0
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Define the custom Huber loss function
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1.0
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Build a simple neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(28 * 28,)),
    Dense(10, activation='softmax')
])

# Compile the model using the custom loss function
model.compile(loss=huber_fn, optimizer=Nadam(), metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {test_accuracy}')

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1688/1688 [==============================] - 21s 9ms/step - loss: 0.0062 - accuracy: 0.9206 - val_loss: 0.0028 - val_accuracy: 0.9642
Epoch 2/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0030 - accuracy: 0.9613 - val_loss: 0.0020 - val_accuracy: 0.9753
Epoch 3/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0022 - accuracy: 0.9731 - val_loss: 0.0019 - val_accuracy: 0.9748
Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0017 - accuracy: 0.9786 - val_loss: 0.0017 - val_accuracy: 0.9772
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0014 - accuracy: 0.9831 - val_loss: 0.0017 - val_accuracy: 0.9780
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0012 - accuracy: 0.9853 - val_loss: 0.0017 - val_accuracy: 0.9772
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - 

#Saving and Loading Models That Contain Custom Components


In [ ]:
import tensorflow as tf
from tensorflow import keras

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [ ]:
# Create a model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model using HuberLoss
model.compile(loss=HuberLoss(threshold=1.5), optimizer='adam', metrics=['accuracy'])

In [ ]:
# Save the model with custom loss function
model.save("my_model_with_custom_loss.h5")

# Load the model with custom loss function
loaded_model = keras.models.load_model(
    "my_model_with_custom_loss.h5",
    custom_objects={"HuberLoss": HuberLoss}
)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loaded_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0064 - accuracy: 0.9170 - val_loss: 0.0028 - val_accuracy: 0.9658
Epoch 2/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0031 - accuracy: 0.9604 - val_loss: 0.0024 - val_accuracy: 0.9702
Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0023 - accuracy: 0.9716 - val_loss: 0.0019 - val_accuracy: 0.9750
Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0018 - accuracy: 0.9781 - val_loss: 0.0017 - val_accuracy: 0.9772
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0015 - accuracy: 0.9823 - val_loss: 0.0019 - val_accuracy: 0.9757
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0012 - accuracy: 0.9851 - val_loss: 0.0019 - val_accuracy: 0.9760
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0011 - accuracy: 0.9875 - val_loss: 0.0016 - val_accuracy

In [ ]:
# Evaluate the model
test_loss, test_accuracy = loaded_model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {test_accuracy}')

#Custom Activation Functions, Initializers, Regularizers, and Constraints

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.nn.relu(weights)

In [ ]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        return {"factor": self.factor}

In [ ]:
# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=my_softplus, kernel_initializer=my_glorot_initializer, kernel_regularizer=my_l1_regularizer),
    keras.layers.Dense(10, activation=my_positive_weights)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Load MNIST data
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 2.8905 - accuracy: 0.3518 - val_loss: 2.1043 - val_accuracy: 0.3973
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 2.0455 - accuracy: 0.4124 - val_loss: 2.0142 - val_accuracy: 0.4168
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9775 - accuracy: 0.4298 - val_loss: 1.9575 - val_accuracy: 0.4346
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9409 - accuracy: 0.4400 - val_loss: 1.9364 - val_accuracy: 0.4429
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 1.9173 - accuracy: 0.4473 - val_loss: 1.9144 - val_accuracy: 0.4494
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9013 - accuracy: 0.4516 - val_loss: 1.8992 - val_accuracy: 0.4477
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 1.8898 - accuracy: 0.4543 - val_loss: 1.8877 - val_accu

#Custom Metrics


In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Iris dataset
iris_data = load_iris()
X = iris_data['data']
y = iris_data['target']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the Huber loss function
def create_huber(threshold):
    def huber_fn(y_true, y_pred):
        error = tf.cast(y_true, dtype=tf.float32) - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        large_error_loss = threshold * (tf.abs(error) - 0.5 * threshold)
        return tf.where(is_small_error, small_error_loss, large_error_loss)
    return huber_fn

# Define the HuberMetric class
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, dtype=tf.float32)  # Convert y_true to float32
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# Build the model with HuberMetric
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(4,)),
    keras.layers.Dense(3, activation='softmax')  # 3 output units for 3 classes with softmax activation
])

# Compile the model with HuberMetric as a custom metric
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy', HuberMetric()])

# Train the model
model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

# Evaluate the model
test_loss, test_acc, test_huber = model.evaluate(X_test_scaled, y_test)
print(f'Test accuracy: {test_acc}, Test Huber: {test_huber}')


Epoch 1/20
4/4 [==============================] - 3s 152ms/step - loss: 1.0649 - accuracy: 0.3000 - huber_metric_5: 1.4219 - val_loss: 0.9827 - val_accuracy: 0.6667 - val_huber_metric_5: 1.5396
Epoch 2/20
4/4 [==============================] - 0s 45ms/step - loss: 0.9772 - accuracy: 0.6917 - huber_metric_5: 1.4224 - val_loss: 0.8943 - val_accuracy: 0.8667 - val_huber_metric_5: 1.5409
Epoch 3/20
4/4 [==============================] - 0s 30ms/step - loss: 0.8964 - accuracy: 0.7750 - huber_metric_5: 1.4248 - val_loss: 0.8145 - val_accuracy: 0.8667 - val_huber_metric_5: 1.5439
Epoch 4/20
4/4 [==============================] - 0s 25ms/step - loss: 0.8246 - accuracy: 0.8000 - huber_metric_5: 1.4294 - val_loss: 0.7441 - val_accuracy: 0.8667 - val_huber_metric_5: 1.5486
Epoch 5/20
4/4 [==============================] - 0s 31ms/step - loss: 0.7610 - accuracy: 0.8083 - huber_metric_5: 1.4357 - val_loss: 0.6827 - val_accuracy: 0.8667 - val_huber_metric_5: 1.5547
Epoch 6/20
4/4 [==================

#Custom Layers


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Define the model using Lambda layer
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Lambda(lambda x: tf.square(x))  # Custom Lambda layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess the MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 20.5242 - accuracy: 0.0000e+00 - val_loss: 20.5419 - val_accuracy: 0.0000e+00
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 20.5242 - accuracy: 0.0000e+00 - val_loss: 20.5419 - val_accuracy: 0.0000e+00
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 20.5242 - accuracy: 0.0000e+00 - val_loss: 20.5419 - val_accuracy: 0.0000e+00
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 20.5243 - accuracy: 0.0000e+00 - val_loss: 20.5419 - val_accuracy: 0.0000e+00
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 20.5419 - accuracy: 0.0000e+00
Test accuracy: 0.0


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Custom Dense Layer
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(tf.matmul(X, self.kernel) + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

# Build the model using MyDense layer
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    MyDense(128, activation='relu'),  # Custom MyDense layer
    keras.layers.Dense(10, activation='softmax')  # Standard Dense layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess the MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2537 - accuracy: 0.9279 - val_loss: 0.1347 - val_accuracy: 0.9594
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1110 - accuracy: 0.9666 - val_loss: 0.1016 - val_accuracy: 0.9711
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0767 - accuracy: 0.9770 - val_loss: 0.0790 - val_accuracy: 0.9756
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0574 - accuracy: 0.9824 - val_loss: 0.0793 - val_accuracy: 0.9768
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0722 - accuracy: 0.9784
Test accuracy: 0.9783999919891357


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Custom Multi-layer
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / (X2 + 1e-6)]  # Avoid division by zero

    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1]

# Example of using MyMultiLayer in a model
input1 = keras.layers.Input(shape=(32,))
input2 = keras.layers.Input(shape=(32,))
output1, output2, output3 = MyMultiLayer()([input1, input2])

# Define a model
model = keras.Model(inputs=[input1, input2], outputs=[output1, output2, output3])

# Compile and train the model (using dummy data for illustration)
model.compile(optimizer='adam', loss='mse')

# Generate dummy data
import numpy as np
X1_train = np.random.rand(100, 32)
X2_train = np.random.rand(100, 32)
Y1_train = X1_train + X2_train
Y2_train = X1_train * X2_train
Y3_train = X1_train / (X2_train + 1e-6)  # Avoid division by zero

# Train the model
model.fit([X1_train, X2_train], [Y1_train, Y2_train, Y3_train], epochs=5, batch_size=32)

# Evaluate the model (using dummy test data)
X1_test = np.random.rand(10, 32)
X2_test = np.random.rand(10, 32)
Y_test = model.predict([X1_test, X2_test])

# Example output shape and results
print("Output shape:", [y.shape for y in Y_test])
print("Sample output values (sum, product, division):")
for i in range(3):
    print(f"Output {i+1}:", Y_test[i][0])


Epoch 1/5
4/4 [==============================] - 1s 6ms/step - loss: 3.2074e-12 - my_multi_layer_loss: 1.7272e-15 - my_multi_layer_1_loss: 2.6022e-16 - my_multi_layer_2_loss: 3.2054e-12
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 3.2074e-12 - my_multi_layer_loss: 1.7272e-15 - my_multi_layer_1_loss: 2.6022e-16 - my_multi_layer_2_loss: 3.2054e-12
Epoch 3/5
4/4 [==============================] - 0s 9ms/step - loss: 3.2074e-12 - my_multi_layer_loss: 1.7272e-15 - my_multi_layer_1_loss: 2.6022e-16 - my_multi_layer_2_loss: 3.2054e-12
Epoch 4/5
4/4 [==============================] - 0s 6ms/step - loss: 3.2074e-12 - my_multi_layer_loss: 1.7272e-15 - my_multi_layer_1_loss: 2.6022e-16 - my_multi_layer_2_loss: 3.2054e-12
Epoch 5/5
1/1 [==============================] - 0s 181ms/step
Output shape: [(10, 32), (10, 32), (10, 32)]
Sample output values (sum, product, division):
Output 1: [1.630543   1.1262277  0.47610322 1.2463055  0.74668926 0.48023456
 0.7119241  0.94170237 0

In [ ]:
import tensorflow as tf
from tensorflow import keras

class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Create a simple model with MyGaussianNoise layer
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    MyGaussianNoise(stddev=0.1),  # Add Gaussian noise during training
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

# Train the model with noisy inputs
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2779 - accuracy: 0.9209 - val_loss: 0.1224 - val_accuracy: 0.9680
Epoch 2/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1226 - accuracy: 0.9639 - val_loss: 0.0954 - val_accuracy: 0.9737
Epoch 3/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0849 - accuracy: 0.9749 - val_loss: 0.1026 - val_accuracy: 0.9718
Epoch 4/5
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0651 - accuracy: 0.9806 - val_loss: 0.0922 - val_accuracy: 0.9725
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0778 - accuracy: 0.9762
Test accuracy: 0.9761999845504761


In [ ]:
import tensorflow as tf
from tensorflow import keras

class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                           kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

# Example of using ResidualBlock in a model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='elu', kernel_initializer='he_normal'),
    ResidualBlock(n_layers=2, n_neurons=128),  # Custom ResidualBlock layer
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess the MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2290 - accuracy: 0.9298 - val_loss: 0.1234 - val_accuracy: 0.9613
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1073 - accuracy: 0.9667 - val_loss: 0.1028 - val_accuracy: 0.9686
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0768 - accuracy: 0.9761 - val_loss: 0.0937 - val_accuracy: 0.9713
Epoch 4/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0602 - accuracy: 0.9799 - val_loss: 0.1082 - val_accuracy: 0.9698
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0996 - accuracy: 0.9721
Test accuracy: 0.972100019454956


#Custom Model

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Define the ResidualBlock class
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                           kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

# Define the ResidualRegressor model
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(3):  # Apply block1 three times
            Z = self.block1(Z)
        for _ in range(3):  # Apply block2 three times
            Z = self.block2(Z)
        return self.out(Z)

# Load and preprocess the California Housing dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing_data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(housing_data.data, housing_data.target, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and compile the ResidualRegressor model
model = ResidualRegressor(output_dim=1)
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.1)

# Evaluate the model
test_loss = model.evaluate(X_test_scaled, y_test)
print(f'Test loss (MSE): {test_loss}')

Epoch 1/10
465/465 [==============================] - 3s 4ms/step - loss: 21.8768 - val_loss: 6.7047
Epoch 2/10
465/465 [==============================] - 1s 3ms/step - loss: 2.3281 - val_loss: 3.9290
Epoch 3/10
465/465 [==============================] - 3s 5ms/step - loss: 3.6640 - val_loss: 0.8711
Epoch 4/10
465/465 [==============================] - 3s 6ms/step - loss: 0.7780 - val_loss: 0.7196
Epoch 5/10
465/465 [==============================] - 4s 8ms/step - loss: 1.7409 - val_loss: 0.7081
Epoch 6/10
465/465 [==============================] - 2s 5ms/step - loss: 0.7386 - val_loss: 0.5964
Epoch 7/10
465/465 [==============================] - 3s 6ms/step - loss: 2.0020 - val_loss: 1.3000
Epoch 8/10
465/465 [==============================] - 2s 5ms/step - loss: 0.9411 - val_loss: 0.6719
Epoch 9/10
465/465 [==============================] - 1s 3ms/step - loss: 2.8853 - val_loss: 0.6098
Epoch 10/10
129/129 [==============================] - 0s 3ms/step - loss: 0.4798
Test loss (MSE): 

#Losses and Metrics Based on Model Internals


In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define the ReconstructingRegressor model
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(64, activation="relu",
                                           kernel_initializer="he_normal") for _ in range(4)]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.1 * recon_loss)  # Adjust regularization factor

        return self.out(Z)


#Computing Gradients Using Autodiff


In [ ]:
def f(w1, w2):
 return 3 * w1 ** 2 + 2 * w1 * w2

In [ ]:
w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [ ]:
(f(w1, w2 + eps) - f(w1, w2)) / eps

10.000000003174137

In [ ]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
 z = f(w1, w2)
gradients = tape.gradient(z, [w1, w2])

In [ ]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]